In [1]:
import sqlite3

conn = sqlite3.connect("reviews.db")

cursor = conn.cursor()

def total_rows(cursor, table_name, print_out=False):
    """ Returns the total number of rows in the database """
    cursor.execute('SELECT COUNT(*) FROM {}'.format(table_name))
    count = cursor.fetchall()
    if print_out:
        print('\nTotal rows: {}'.format(count[0][0]))
    return count[0][0]

total_rows(cursor, "REVIEWS")

OperationalError: no such table: REVIEWS

In [3]:
import pandas as pd
df = pd.read_sql_query("SELECT * from REVIEWS", conn)

In [ ]:
#df.groupby(df.columns.tolist(),as_index=False).size()

duplicateRowsDF = df[df.duplicated()]
print(duplicateRowsDF)

In [5]:
df.head()


,NODE,URL,bTEXT,REVIEWBODY,REVIEWRATING,ITEMREVIEWED,REVIEWASPECT,REVIEWAUTHOR,REVIEWDESCR
0,_:node2c2e3ffd705962c891db9ab89a44cacc,http://20te20.ir/527007-%D8%AA%D9%86%D9%88%D8%...,None,None,<http://20te20.ir/527007-%D8%AA%D9%86%D9%88%D8...,None,None,None,None
1,_:node1f865f4bfa138ac184ddbbd383299621,http://67kmm.com/~hello-neighbor/android,None,"""Hello Neighbor est un jeu d'horreur \u00C3\u0...",None,None,None,None,None
2,_:node7d8efbe93e9573eb609b41cfa3be84,http://67kmm.com/~snapchat/android,None,"""Bon7Snapchat est une application de partage i...",None,None,None,None,None
3,_:node98788ddf61c5e4913d4ee048868b02,http://20te20.ir/517230-%D8%A7%D8%AA%D9%88-%D8...,None,None,<http://20te20.ir/517230-%D8%A7%D8%AA%D9%88-%D...,None,None,None,None
4,_:nodebb685880f9d030fca1b194c14548d8b5,http://67kmm.com/~grand-theft-auto-san-andreas...,None,"""Par Matt Mullarkey-TonerExcellent9Du haut de ...",None,None,None,_:node55c1dea05a3a6929edebabe6265ba3b,None


In [ ]:
perc =[.20, .40, .60, .80] 
include =['object', 'float', 'int']
desc = df.describe(percentiles = perc, include = include) 


# Data Framing and Subsetting Schema.org

In [ ]:
df = pd.read_csv("./reviews_all.csv", sep="\n", engine="python", error_bad_lines=False, delimiter=',')

In [ ]:
reviewsDF = spark.read.csv('../Data/reviews_all.csv', header=True, sep="\t")
print("total:" + str(reviewsDF.count()))
reviewsDF = reviewsDF.filter(reviewsDF.REVIEWBODY.isNotNull())
print("without text:" + str(reviewsDF.count()))
reviewsDF = reviewsDF.filter(reviewsDF.REVIEWRATING.isNotNull())
print("without rating:" + str(reviewsDF.count()))
reviewsDF.head(5)

In [ ]:
from langdetect import DetectorFactory
DetectorFactory.seed = 0

def detectLang(x):
    try:
        return detect(x)
    except:
        print("exception:", x)
    else: 
        print("sad:", x)
detect_lang = udf(detectLang, StringType())
df = reviewsDF.withColumn("LANG", lit(detect_lang(reviewsDF.REVIEWBODY)))
reviews_engDF = df.filter(df.LANG == "en")
print("with eng:" + str(reviews_engDF.count()))

In [1]:
from bs4 import BeautifulSoup
from requests import get
import json
import random
import string
import sqlite3
import logging
import re

In [2]:
def generateNode(length):
    letters_and_digits = string.ascii_letters + string.digits
    result_str = ''.join((random.choice(letters_and_digits) for i in range(length)))
    node = "_:znode" + result_str
    return node
    # print("Random alphanumeric String is:", result_str)
# generateNode(31)

In [8]:
host = 'https://www.imdb.com'
# https://www.imdb.com/search/title/?title_type=tv_movie,tv_episode&release_date=2000-01-01,2020-12-31&user_rating=1.0,10.0&languages=en&start=1&ref_=adv_nxt
# +50
# https://www.imdb.com/search/title/?title_type=tv_movie,tv_episode&release_date=2000-01-01,2020-12-31&user_rating=1.0,10.0&languages=en&start=51&ref_=adv_nxt
page_count = 0
movie_count = 0
review_count = 0
# umstellen auf 1001
for page in range (0,11):
    page_count= page*50+1
    # print(page_count)
    url = f'https://www.imdb.com/search/title/?title_type=tv_movie,tv_episode&release_date=2000-01-01,2020-12-31&user_rating=1.0,10.0&languages=en&start={page_count}&ref_=adv_nxt'
    response = get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    movie_containers = html_soup.find_all('div', class_ = 'lister-item mode-advanced')
    # print(type(movie_containers))
    # print(len(movie_containers))
    i = 0
    for movie in movie_containers:
        # link to site with all review previews for each movie
        link = 'https://www.imdb.com/' + movie_containers[i].a['href'] + 'reviews'
        # link = 'https://www.imdb.com//title/tt8022928/reviews'
        print(link)
        movie_count += 1
        i += 1
        review_collection = BeautifulSoup(get(link).text, 'html.parser').body.find_all('div', class_='actions text-muted')
        for tiles in review_collection:
            permalinks = tiles.find_all('a', href=True, text='Permalink')
            for permalink in permalinks:
                review_link = host + permalink['href']
                # review_link = "https://www.imdb.com/review/rw3754733/"
                # review_link = "https://www.imdb.com/review/rw5040450/"
                print(review_link)
                review_count += 1
                review_soup = BeautifulSoup(get(review_link).text, 'html.parser')
                try:
                    warning = review_soup.body.find_all(string=re.compile('.*{0}.*'.format('Warning: Spoilers')), recursive=True)
                except:
                    pass
                container = review_soup.find("script",type="application/ld+json")
                for item in container:
                    if item.__contains__('http://schema.org'):
                        text = str(item.extract())
                        oJson = json.loads(text)
                        # print(oJson)
                        if (warning):
                            # ("WARNING!")
                            reviewBodyContainer = review_soup.body.find_all('div', class_="text show-more__control")
                            for item in reviewBodyContainer:
                                reviewBody = item.text
                            # print(reviewBodyContainer)
                            # reviewBody = reviewBodyContainer[0].text
                        else:
                            reviewBody = str(oJson['reviewBody'])
                        try:
                            node = generateNode(31)
                            reviewRating = str(oJson['reviewRating'])
                            worstRating = str(oJson['reviewRating']['worstRating'])
                            bestRating = str(oJson['reviewRating']['bestRating'])
                            ratingValue = str(oJson['reviewRating']['ratingValue'])
                        except: 
                            continue
                        print('node:' + node)
                        print('url: '+ review_link)
                        print('reviewBody: ' + reviewBody)
                        print('worstRating: ' + worstRating)
                        print('bestRating: ' + bestRating)
                        print('ratingValue: ' + ratingValue)
                            
                        # print("INSERT OR IGNORE INTO MOVIEREVIEWS (NODE, URL, REVIEWBODY, RATING, REVIEWRATING, BESTRATING, WORSTRATING) VALUES (?,?,?,?,?,?,?);""",(node,url,reviewBody,reviewRating,ratingValue,bestRating,worstRating))
                        # c.execute("INSERT OR IGNORE INTO moviereviews (NODE, URL, REVIEWBODY, RATING, REVIEWRATING, BESTRATING, WORSTRATING) VALUES (?,?,?,?,?,?,?);",(node,url,reviewBody,reviewRating,ratingValue,bestRating,worstRating))
                        # conn.commit()
logging.debug("number of reviews: " + str(review_count))

https://www.imdb.com//title/tt8022928/reviews
https://www.imdb.com/review/rw5037147/
node:_:znodexf0Op3xtlK4jm9sHatMIEzKrjvIuH1h
url: https://www.imdb.com/review/rw5037147/
reviewBody: Descendants franchise never disappoint, With the 3rd and last installment,Descendants never fail to entertain and touch us! The loss of Cameron Boyce made the movie sad but still the movie was very fun to watch with great songs and the same atmosphere of the first 2 movies! So sad it ends here.WE WILL NEVER FORGET YOU * CAMERON BOYCE*
worstRating: 1
bestRating: 10
ratingValue: 6
https://www.imdb.com/review/rw5036852/
node:_:znodeW5r9rhpgKPPxMIxZZNb2zmuoj9U1u9R
url: https://www.imdb.com/review/rw5036852/
reviewBody: The movie was great, really great, but what did it for me was that touching moment honoring Cameron Boyce. Truly magnificent.
worstRating: 1
bestRating: 10
ratingValue: 10
https://www.imdb.com/review/rw5036439/
node:_:znodeVwubH7uLt2m6RsJGMe8EeIxNfFXcRH6
url: https://www.imdb.com/review/rw5036

node:_:znodebKK2P2IXCQfqNNvqtOecrAVTPoXA5tL
url: https://www.imdb.com/review/rw5036712/
reviewBody: From The story that has underlying themes in regards to Immigration to the acting, cinematography and effects, I found it hard to believe this was a film that came out of disney channel. Easily the best movie out of the trilogy. The songs are better, the dancing is better, the stakes are better. Kenny Ortega concluded yet another trilogy perfectly
worstRating: 1
bestRating: 10
ratingValue: 10
https://www.imdb.com/review/rw5037120/
node:_:znodedSaYiPXYjauytEUMZXIJ3g7Q7uSZrAl
url: https://www.imdb.com/review/rw5037120/
reviewBody: Love love loved D3!!! What a perfect ending for the trilogy!Mr Ortega made sure the messages of "family," "acceptance," "second chances," and "celebrating our differences" came ringing through with fabulous music and dancing that was a lot of fun to watch -a lot of work for the actors too, I bet - but so worth it for the viewers.It was great to see Ben as the Bea

node:_:znodeNbPVIdyihyVrU4H86VzKly77ThJL40c
url: https://www.imdb.com/review/rw5041776/
reviewBody: I am a huge fan of Disney so this may be biased but the movie is great. It's an amazing conclusion to the franchise and the songs and acting are really good for a TV movie. Loved it, fun watch.
worstRating: 1
bestRating: 10
ratingValue: 9
https://www.imdb.com/review/rw5040686/
node:_:znodeaFexTzST60rmhVDdgnr6tdSTap2XKQE
url: https://www.imdb.com/review/rw5040686/
reviewBody: They did a great job with the plot and music! References to the other films and easter eggs were nicely done! The dancing was great! Great cast as usual! Great messages like the other films and centered around unity, teamwork, understanding people, honesty, and helping others. This is a movie that can be watched by all and the franchise connects all of the Disney movies.
worstRating: 1
bestRating: 10
ratingValue: 10
https://www.imdb.com/review/rw5040450/
Error
https://www.imdb.com/review/rw5053845/
node:_:znodeSBqdMn

node:_:znodeT3JprITTIZbHa1YaYHDlAAa7jyKZO62
url: https://www.imdb.com/review/rw5036852/
reviewBody: The movie was great, really great, but what did it for me was that touching moment honoring Cameron Boyce. Truly magnificent.
worstRating: 1
bestRating: 10
ratingValue: 10
https://www.imdb.com/review/rw5036439/
node:_:znodewMOg2ugRyfA6j3opBcj8eGZeh673ldn
url: https://www.imdb.com/review/rw5036439/
reviewBody: I mean, it's a Disney Channel Original trilogy, so obviously it's not going to be perfect. But it's clear that a lot of thought and love went into this movie - definitely much more than I expected. I'm beyond satisfied right now.
worstRating: 1
bestRating: 10
ratingValue: 10
https://www.imdb.com/review/rw5036404/
node:_:znodemOm2HD4X5lwG5ivZ676arE8NKFHoVkB
url: https://www.imdb.com/review/rw5036404/
reviewBody: I am so glad Disney makes such well written and family friendly movies available to watch on the Disney channel! Awesome movie with a great message!
worstRating: 1
bestRating

node:_:znode29NO5sr9CyjdUzxUegCTuoMuBgKGaIZ
url: https://www.imdb.com/review/rw5036712/
reviewBody: From The story that has underlying themes in regards to Immigration to the acting, cinematography and effects, I found it hard to believe this was a film that came out of disney channel. Easily the best movie out of the trilogy. The songs are better, the dancing is better, the stakes are better. Kenny Ortega concluded yet another trilogy perfectly
worstRating: 1
bestRating: 10
ratingValue: 10
https://www.imdb.com/review/rw5037120/
node:_:znodeqlA41E8cU7tKJtBI9lBUlKWxX0cKj1z
url: https://www.imdb.com/review/rw5037120/
reviewBody: Love love loved D3!!! What a perfect ending for the trilogy!Mr Ortega made sure the messages of "family," "acceptance," "second chances," and "celebrating our differences" came ringing through with fabulous music and dancing that was a lot of fun to watch -a lot of work for the actors too, I bet - but so worth it for the viewers.It was great to see Ben as the Bea

KeyboardInterrupt: 